In [4]:
"""
Post-processing for segmentation predictions (TIFF in ➜ TIFF/PNG out)

What it does
------------
- Reads a multi-page TIFF of predictions where pixel values are {0,85,170,255}
  (0=boundary, 1/2=cells, 3=white) and some pages may be completely black.
- Detects fully-black frames and leaves them black at the same indices in the outputs.
- Applies the original post-processing pipeline to non-black frames (logic preserved).
- Exports:
    1) post_processed_085.tif  — multi-page TIFF with values {0,85,170,255}
    2) post_processed_0_3.tif  — multi-page TIFF with class indices {0,1,2,3}
    3) frames_0_3.npz          — NumPy archive with array 'frames' of shape (N, H, W)
    4) post_processed_png/     — folder of per-frame PNGs (values {0,85,170,255})

Requirements
------------
Python >= 3.9
pip install numpy opencv-python scipy scikit-image tifffile tqdm
"""

# =========================
# USER CONFIG (EDIT HERE)
# =========================
PRED_TIFF_PATH = "/content/drive/MyDrive/Final_Project/output_predictions/all_predictions.tif"  # input predictions TIFF
OUTPUT_DIR     = "/content/drive/MyDrive/Final_Project/postprocess_out"                         # output folder

# Optional: limit number of frames processed (None = all)
MAX_FRAMES = None

# Progress / logging
VERBOSE_PER_FRAME = False   # if True, prints per-frame info line (kernel/min sizes/etc.)

# =========================
# DERIVED PATHS (no edit)
# =========================
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_TIF_085_PATH = os.path.join(OUTPUT_DIR, "post_processed_085.tif")  # values {0,85,170,255}
OUTPUT_TIF_03_PATH  = os.path.join(OUTPUT_DIR, "post_processed_0_3.tif")  # values {0,1,2,3}
OUTPUT_03_NPZ_PATH  = os.path.join(OUTPUT_DIR, "frames_0_3.npz")          # npz with array 'frames'
OUTPUT_PNG_DIR      = os.path.join(OUTPUT_DIR, "post_processed_png")      # per-frame PNGs (0..255)

# =========================
# IMPORTS
# =========================
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import cv2
import numpy as np
import tifffile as tiff
from tqdm import tqdm

from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.morphology import remove_small_objects, closing, opening, square, skeletonize
from skimage.measure import label, regionprops

# =========================
# Constants & Small Utils
# =========================
VALS = np.array([0, 85, 170, 255], dtype=np.uint8)  # 0=boundary, 1/2=cells, 3=white

def is_all_black(u8: np.ndarray) -> bool:
    """Return True if the image is non-empty and all zeros."""
    return u8 is not None and u8.size > 0 and (u8.max() == 0)

def png_to_classes(img_u8: np.ndarray) -> np.ndarray:
    """
    Map grayscale {0,85,170,255} (or {0,1,2,3}) to class indices {0,1,2,3}.
    Args:
        img_u8: 2D uint8 image
    Returns:
        2D uint8 class map in {0,1,2,3}
    """
    uniq = np.unique(img_u8)
    if set(uniq.tolist()).issubset({0,1,2,3}):
        return img_u8.astype(np.uint8)
    out = np.zeros_like(img_u8, dtype=np.uint8)
    for c, v in enumerate(VALS):
        out[img_u8 == v] = c
    return out

def classes_to_png(cls: np.ndarray) -> np.ndarray:
    """
    Map class indices {0,1,2,3} to grayscale {0,85,170,255}.
    Args:
        cls: 2D uint8 class map (0..3)
    Returns:
        2D uint8 image with values in {0,85,170,255}
    """
    out = np.zeros_like(cls, dtype=np.uint8)
    for c, v in enumerate(VALS):
        out[cls == c] = v
    return out

def touches_border(mask_bool: np.ndarray) -> bool:
    """Return True if a boolean mask touches any image border."""
    return mask_bool[0,:].any() or mask_bool[-1,:].any() or mask_bool[:,0].any() or mask_bool[:,-1].any()

# =========================
# Scale Estimation
# =========================
def estimate_cell_scale(cls: np.ndarray):
    """
    Estimate morphological/area scales from a class map.
    Args:
        cls: 2D uint8 class map (0..3)
    Returns:
        kernel: int, morphological kernel size (square)
        min_cell: int, minimal connected component area for 'cell' (classes 1 or 2)
        min_white: int, minimal connected component area for 'white' (class 3)
        repeats: int, how many close-open cycles
        r_med: float, median distance-transform radius at peaks
    """
    cell_mask = ((cls == 1) | (cls == 2)).astype(np.uint8)
    if cell_mask.sum() == 0:
        return 7, 250, 1200, 1, 6.0

    dist = cv2.distanceTransform(cell_mask, cv2.DIST_L2, 5)
    peaks = peak_local_max(dist, min_distance=5, labels=cell_mask)
    radii = dist[peaks[:, 0], peaks[:, 1]] if peaks.size else np.array([], dtype=np.float32)
    radii = radii[radii > 0]
    r_med = float(np.median(radii)) if radii.size else 6.0
    d_med = 2.0 * r_med

    kernel    = max(3, int(round(d_med / 4)))
    min_cell  = max(80, int(round(np.pi * (0.6 * r_med) ** 2)))
    min_white = int(max(800, 4 * min_cell))
    repeats   = 1
    return kernel, min_cell, min_white, repeats, r_med

# =========================
# Core post-processing (single frame)
# =========================
def process_frame_array(img_u8: np.ndarray, verbose: bool = False):
    """
    Process a single prediction frame (uint8, values {0,85,170,255} or {0..3}).

    Args:
        img_u8: 2D uint8 input prediction frame
        verbose: if True, prints per-frame scale info

    Returns:
        stage: 2D uint8 class map after post-processing (values {0,1,2,3})
    """
    # Short-circuit: keep black frames black (no processing)
    if is_all_black(img_u8):
        return np.zeros_like(img_u8, dtype=np.uint8)

    # Map to classes 0..3
    cls = png_to_classes(img_u8)

    # Estimate per-frame scales
    KERNEL, MIN_CELL_SIZE, MIN_WHITE_SIZE, REPEATS, r_med = estimate_cell_scale(cls)
    if verbose:
        print(f"[INFO] KERNEL={KERNEL}, MIN_CELL={MIN_CELL_SIZE}, MIN_WHITE={MIN_WHITE_SIZE}, REPEATS={REPEATS}, r_med={r_med:.2f}")

    # ----- Stage 1: cleanup -----
    is_boundary = (cls == 0)
    is_white    = (cls == 3)
    is_cell     = (cls == 1) | (cls == 2)

    cell_clean = remove_small_objects(label(is_cell), min_size=MIN_CELL_SIZE) > 0
    for _ in range(REPEATS):
        cell_clean = closing(cell_clean, square(KERNEL))
        cell_clean = opening(cell_clean, square(KERNEL))

    white_clean = remove_small_objects(label(is_white), min_size=MIN_WHITE_SIZE) > 0
    for _ in range(REPEATS):
        white_clean = closing(white_clean, square(KERNEL))
        white_clean = opening(white_clean, square(KERNEL))

    stage = np.zeros_like(cls, dtype=np.uint8)
    stage[cell_clean & (cls == 1)] = 1
    stage[cell_clean & (cls == 2)] = 2
    stage[white_clean]             = 3
    stage[is_boundary]             = 0

    # ----- Stage 3: watershed + majority -----
    cell_mask = ((stage == 1) | (stage == 2)).astype(np.uint8)
    distance  = cv2.distanceTransform(cell_mask, cv2.DIST_L2, 5)
    min_dist  = max(3, int(round(r_med/2)))
    lm = peak_local_max(distance, min_distance=min_dist, labels=cell_mask)
    lm_mask = np.zeros_like(cell_mask, dtype=bool)
    if lm.size:
        lm_mask[tuple(lm.T)] = True
    markers  = ndi.label(lm_mask)[0]
    labels_ws = watershed(-distance, markers, mask=cell_mask)

    stage3 = stage.copy()
    for reg in regionprops(labels_ws):
        yyx = reg.coords
        vals = stage[yyx[:,0], yyx[:,1]]
        vals = vals[vals != 0]
        maj  = np.bincount(vals).argmax() if vals.size else 3
        stage3[yyx[:,0], yyx[:,1]] = maj
    stage3[(stage == 3)] = 3
    stage = stage3

    # Save references for stage 9
    labels_ws3 = labels_ws.astype(np.int32)
    sk3 = skeletonize((stage == 0)).astype(np.uint8)

    # ----- Stage 3.5: remove tiny boundaries -----
    b_mask = (stage == 0)
    lab_b = label(b_mask)
    small_thr = max(8, int(round(0.10*np.pi*(r_med**2))))
    stage35 = stage.copy()
    for reg in regionprops(lab_b):
        if reg.area <= small_thr:
            m = np.zeros_like(b_mask, dtype=bool)
            yy, xx = reg.coords[:,0], reg.coords[:,1]
            m[yy,xx] = True
            ring = cv2.dilate(m.astype(np.uint8), np.ones((3,3),np.uint8), 1).astype(bool) & ~m
            neigh = stage35[ring]; neigh = neigh[neigh != 0]
            stage35[m] = np.bincount(neigh).argmax() if neigh.size else 3
    stage = stage35

    # ----- Stage 4 -----
    boundary = (stage == 0).astype(np.uint8)
    cells    = ((stage == 1) | (stage == 2)).astype(np.uint8)
    white0   = (stage == 3)
    WIN = max(11, int(round(3*r_med)) | 1)
    bdens = cv2.boxFilter(boundary.astype(np.float32), -1, (WIN,WIN), normalize=True)
    cfrac = cv2.boxFilter(cells.astype(np.float32),    -1, (WIN,WIN), normalize=True)
    dist_c = cv2.distanceTransform(cells, cv2.DIST_L2, 5)
    lowc = (dist_c < 0.5*r_med).astype(np.uint8)
    lowc_frac = cv2.boxFilter(lowc.astype(np.float32), -1, (WIN,WIN), normalize=True)
    T_BD, T_CELL, T_LOWC = 0.30, 0.45, 0.60
    noisy = (bdens > T_BD) & (cfrac < T_CELL) & (lowc_frac > T_LOWC)
    k = max(3, (WIN//3) | 1)
    noisy = cv2.morphologyEx((noisy.astype(np.uint8)*255), cv2.MORPH_OPEN, np.ones((k,k),np.uint8)) > 0
    noisy &= ~(stage == 0)
    stage4 = stage.copy(); stage4[noisy] = 3; stage4[white0] = 3
    stage = stage4

    # ----- Stage 5 -----
    b_mask = (stage == 0).astype(np.uint8)
    skel = skeletonize(b_mask > 0).astype(np.uint8)
    thick = (b_mask > 0) & (~skel.astype(bool))
    k2 = np.array([[1,1],[1,1]], dtype=np.uint8)
    blk2 = cv2.filter2D(b_mask, -1, k2) == 4
    thick |= blk2
    lab = label(thick.astype(np.uint8), connectivity=2)
    THICK_MIN_AREA, SKEL_RATIO_MAX = 5, 0.65
    stage5 = stage.copy()
    for reg in regionprops(lab):
        area = reg.area
        if area < THICK_MIN_AREA:
            continue
        comp = np.zeros_like(b_mask, dtype=bool)
        yy, xx = reg.coords[:,0], reg.coords[:,1]
        comp[yy, xx] = True
        sk = skeletonize(comp).astype(np.uint8).sum()
        ratio = sk / float(area) if area > 0 else 1.0
        has_2x2 = (cv2.filter2D(comp.astype(np.uint8), -1, k2) == 4).any()
        if (ratio < SKEL_RATIO_MAX) or has_2x2:
            ring = cv2.dilate(comp.astype(np.uint8), np.ones((3,3), np.uint8), 1).astype(bool) & ~comp
            neigh = stage[ring]; neigh = neigh[neigh != 0]
            if neigh.size:
                cells_nei = neigh[(neigh == 1) | (neigh == 2)]
                if cells_nei.size:
                    repl = 1 if (cells_nei == 1).sum() >= (cells_nei == 2).sum() else 2
                else:
                    repl = 3
            else:
                repl = 1
            stage5[comp] = repl
    stage = stage5

    # ----- Stage 6 -----
    b_mask   = (stage == 0).astype(np.uint8)
    cells    = ((stage == 1) | (stage == 2)).astype(np.uint8)
    non_bd   = (b_mask == 0).astype(np.uint8)
    dist_to_bd = cv2.distanceTransform(non_bd, cv2.DIST_L2, 5)
    TAU = max(int(round(2.0 * r_med)), 6)
    far_from_bd = dist_to_bd > TAU
    WIN = max(21, int(round(5 * r_med)) | 1)
    bdens = cv2.boxFilter(b_mask.astype(np.float32), -1, (WIN, WIN), normalize=True)
    LOW_BD = 0.05
    low_bd = bdens < LOW_BD
    candidates = (cells > 0) & far_from_bd & low_bd
    CELL_AREA_EST   = float(np.pi * max(r_med, 3.0)**2)
    MIN_BIG_CELLS   = 15
    BIG_AREA_THRESH = int(MIN_BIG_CELLS * CELL_AREA_EST)
    lab = label(candidates, connectivity=2)
    areas = np.bincount(lab.ravel())
    areas = np.pad(areas, (0, max(0, lab.max()+1 - areas.size)))
    big_mask = (lab > 0) & (areas[lab] >= BIG_AREA_THRESH)
    stage6 = stage.copy(); stage6[big_mask] = 3
    stage = stage6

    # ----- Stage 7 -----
    CELL_AREA_EST   = float(np.pi * max(r_med, 3.0)**2)
    MAX_COMP_AREA   = int(5 * CELL_AREA_EST)
    T_RING_WHITE    = 0.98
    WHITE_WIN       = max(15, int(round(2.2 * r_med)) | 1)
    T_LOCAL_WHITE   = 0.75
    FUSE_FRAC       = 0.03
    cells_mask = ((stage == 1) | (stage == 2)).astype(np.uint8)
    lab_cells = label(cells_mask, connectivity=2)
    white_mask = (stage == 3).astype(np.uint8)
    labw = label(white_mask, connectivity=2)
    bg_ids = set(np.unique(labw[0, :]).tolist() +
                 np.unique(labw[-1, :]).tolist() +
                 np.unique(labw[:, 0]).tolist() +
                 np.unique(labw[:, -1]).tolist())
    bg_ids.discard(0)
    bg_white = np.isin(labw, list(bg_ids))
    inner_white = (white_mask.astype(bool) & (~bg_white))
    RING_K1 = max(9, int(round(0.8 * r_med)) | 1)
    RING_K2 = max(RING_K1+2, int(round(1.5 * r_med)) | 1)
    ker1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (RING_K1, RING_K1))
    ker2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (RING_K2, RING_K2))
    white_frac_local = cv2.boxFilter((stage == 3).astype(np.float32), -1, (WHITE_WIN, WHITE_WIN), normalize=True)

    to_white_A = np.zeros_like(cells_mask, dtype=bool)
    to_white_B = np.zeros_like(cells_mask, dtype=bool)
    for reg in regionprops(lab_cells):
        if reg.area > MAX_COMP_AREA:
            continue
        comp = np.zeros_like(cells_mask, dtype=bool)
        yy, xx = reg.coords[:, 0], reg.coords[:, 1]
        comp[yy, xx] = True
        ring1 = cv2.dilate(comp.astype(np.uint8), ker1, 1).astype(bool) & ~comp
        ring1_nb = ring1 & (stage != 0)
        if np.any(ring1_nb):
            if np.mean((stage[ring1_nb] == 3).astype(np.float32)) >= 0.98 and np.mean(inner_white[ring1_nb]) >= 0.95:
                if not touches_border(cv2.dilate(comp.astype(np.uint8), ker1, 1).astype(bool)):
                    to_white_A |= comp
                    continue
        ring2 = cv2.dilate(comp.astype(np.uint8), ker2, 1).astype(bool) & ~comp
        ring2_nb = ring2 & (stage != 0)
        if np.any(ring2_nb):
            if np.all(stage[ring2_nb] == 3) and (white_frac_local[comp].mean() >= T_LOCAL_WHITE):
                if not touches_border(cv2.dilate(comp.astype(np.uint8), ker2, 1).astype(bool)):
                    to_white_B |= comp
    to_white_comp = to_white_A | to_white_B
    total_pixels = int(to_white_comp.sum())
    limit_pixels = int(FUSE_FRAC * stage.size)
    if total_pixels > limit_pixels:
        labs = label(to_white_comp, connectivity=2)
        areas = np.bincount(labs.ravel())
        order = np.argsort(areas)[1:]
        chosen = np.zeros_like(to_white_comp, dtype=bool)
        acc = 0
        for k in order:
            comp = (labs == k)
            s = int(comp.sum())
            if acc + s > limit_pixels:
                continue
            chosen |= comp
            acc += s
            if acc >= limit_pixels:
                break
        to_white_comp = chosen
    stage7 = stage.copy(); stage7[to_white_comp] = 3
    stage = stage7

    # ----- Stage 8 -----
    nonwhite = (stage != 3).astype(np.uint8)
    lab_nonw = label(nonwhite, connectivity=2)
    RING_K2b = max(5, int(round(10 * r_med)) | 1)
    ker2b = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (RING_K2b, RING_K2b))
    to_white_any = np.zeros_like(nonwhite, dtype=bool)
    for reg in regionprops(lab_nonw):
        comp = np.zeros_like(nonwhite, dtype=bool)
        yy, xx = reg.coords[:,0], reg.coords[:,1]
        comp[yy, xx] = True
        if touches_border(comp):
            continue
        ring = cv2.dilate(comp.astype(np.uint8), ker2b, 1).astype(bool) & ~comp
        if not np.any(ring):
            continue
        ring_vals = stage[ring]
        white_ratio = np.mean(ring_vals == 3)
        if white_ratio >= 0.98:
            to_white_any |= comp
    stage8 = stage.copy(); stage8[to_white_any] = 3
    stage = stage8

    # ----- Stage 9: gentle boundary restoration -----
    b_now      = (stage == 0).astype(np.uint8)
    cells_mask = ((stage == 1) | (stage == 2)).astype(np.uint8)
    white_mask = (stage == 3).astype(np.uint8)
    cand = (sk3 > 0) & (b_now == 0)
    lab_cand = label(cand, connectivity=2)

    MIN_LEN        = max(6,  int(round(0.8 * r_med)))
    MAX_LEN        = max(20, int(round(3.0 * r_med)))
    T_CELL_FRAC    = 0.65
    T_WHITE_FRAC   = 0.20
    KREC           = max(7,  int(round(1.1 * r_med)) | 1)
    ker_rec        = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (KREC, KREC))
    LWIN_WHITE     = max(13, int(round(2.0 * r_med)) | 1)
    T_WHITE_LOCAL  = 0.45
    ANCH_R         = max(2,  int(round(0.6 * r_med)))
    MAX_PCT_GLOBAL = 0.0015

    white_frac_local = cv2.boxFilter(white_mask.astype(np.float32), -1,
                                     (LWIN_WHITE, LWIN_WHITE), normalize=True)
    ker_anchor = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*ANCH_R+1, 2*ANCH_R+1))
    anchor = cv2.dilate(b_now, ker_anchor, 1).astype(bool)

    def endpoints_mask(skel_bool: np.ndarray) -> np.ndarray:
        u8 = skel_bool.astype(np.uint8)
        nb = cv2.filter2D(u8, -1, np.ones((3,3), np.uint8))
        return (u8 > 0) & ((nb - u8) == 1)

    to_restore = np.zeros_like(cand, dtype=bool)
    for reg in regionprops(lab_cand):
        comp = np.zeros_like(cand, dtype=bool)
        yy, xx = reg.coords[:, 0], reg.coords[:, 1]
        comp[yy, xx] = True
        comp_skel = skeletonize(comp).astype(bool)
        L = int(comp_skel.sum())
        if L < MIN_LEN or L > MAX_LEN:
            continue
        if white_frac_local[comp_skel].mean() > T_WHITE_LOCAL:
            continue
        ring = cv2.dilate(comp_skel.astype(np.uint8), ker_rec, 1).astype(bool) & ~comp_skel
        if not np.any(ring):
            continue
        tot_nb     = max(1, int(ring.sum()))
        cell_frac  = int((ring & (cells_mask > 0)).sum()) / tot_nb
        white_frac = int((ring & (white_mask > 0)).sum()) / tot_nb
        if cell_frac < T_CELL_FRAC or white_frac > T_WHITE_FRAC:
            continue
        ends = endpoints_mask(comp_skel)
        if (ends & anchor).sum() < 2:
            continue
        to_restore |= comp_skel

    limit_pixels = int(MAX_PCT_GLOBAL * stage.size)
    if to_restore.sum() > limit_pixels:
        lab = label(to_restore, connectivity=2)
        segs = [(int(skeletonize((lab == k)).sum()), (lab == k)) for k in range(1, lab.max()+1)]
        segs.sort(key=lambda t: t[0])  # prefer shorter segments
        picked = np.zeros_like(to_restore, dtype=bool)
        acc = 0
        for L, m in segs:
            s = int(m.sum())
            if acc + s > limit_pixels:
                continue
            picked |= m
            acc += s
            if acc >= limit_pixels:
                break
        to_restore = picked

    stage9 = stage.copy()
    stage9[to_restore] = 0
    stage = stage9

    return stage  # 0..3

# =========================
# TIFF pipeline (batch)
# =========================
def postprocess_tiff_to_outputs():
    """
    Full batch pipeline:
      - Read all pages from PRED_TIFF_PATH (limit by MAX_FRAMES if set).
      - Collect black indices; process only non-black pages.
      - Assemble outputs maintaining original indices:
          * post_processed_0_3.tif  (0..3)
          * post_processed_085.tif  (0,85,170,255)
          * frames_0_3.npz          (array 'frames' with shape (N,H,W))
          * post_processed_png/     (per-frame PNGs, values {0,85,170,255})
    """
    with tiff.TiffFile(PRED_TIFF_PATH) as tif:
        total_pages = len(tif.pages)
        if total_pages == 0:
            raise RuntimeError(f"No pages found in {PRED_TIFF_PATH}")
        N = total_pages if MAX_FRAMES is None else min(MAX_FRAMES, total_pages)

        # Read all pages as uint8
        preds_u8 = [tif.pages[i].asarray().astype(np.uint8) for i in range(N)]

    # Detect black indices
    black_indices = {i for i, img in enumerate(preds_u8) if is_all_black(img)}

    # Process non-black with a single progress bar
    processed_0_3 = []  # per-index list, will fill with 0..3 arrays
    H, W = preds_u8[0].shape  # assume consistent dims

    pbar = tqdm(range(N), desc="Post-processing TIFF", unit="frame")
    for i in pbar:
        img = preds_u8[i]
        if i in black_indices:
            processed_0_3.append(np.zeros((H, W), dtype=np.uint8))
            continue
        processed_0_3.append(process_frame_array(img, verbose=VERBOSE_PER_FRAME))

    # Stack arrays
    arr_0_3  = np.stack(processed_0_3, axis=0).astype(np.uint8)                  # (N,H,W) in {0..3}
    arr_085  = np.stack([classes_to_png(f) for f in processed_0_3], axis=0).astype(np.uint8)  # (N,H,W)

    # Save TIFFs
    tiff.imwrite(OUTPUT_TIF_03_PATH,  arr_0_3)
    tiff.imwrite(OUTPUT_TIF_085_PATH, arr_085)

    # Save NPZ list/array of 0..3 frames
    np.savez_compressed(OUTPUT_03_NPZ_PATH, frames=arr_0_3)

    # Save per-frame PNGs (values {0,85,170,255})
    os.makedirs(OUTPUT_PNG_DIR, exist_ok=True)
    pad = len(str(N-1))  # zero-padding width
    for i in range(N):
        fname = f"frame_{i:0{pad}d}.png"
        cv2.imwrite(os.path.join(OUTPUT_PNG_DIR, fname), arr_085[i])

    print(f"\n✅ Saved 0..3 TIFF:              {OUTPUT_TIF_03_PATH}")
    print(f"✅ Saved 0/85/170/255 TIFF:      {OUTPUT_TIF_085_PATH}")
    print(f"✅ Saved NPZ (0..3 frames):      {OUTPUT_03_NPZ_PATH}")
    print(f"✅ Saved per-frame PNGs in dir:  {OUTPUT_PNG_DIR}")

# =========================
# Main
# =========================
if __name__ == "__main__":
    postprocess_tiff_to_outputs()


Post-processing TIFF:   0%|          | 0/10 [00:00<?, ?frame/s]

[INFO] KERNEL=8, MIN_CELL=282, MIN_WHITE=1128, REPEATS=1, r_med=15.79


Post-processing TIFF:  10%|█         | 1/10 [00:54<08:09, 54.42s/frame]

[INFO] KERNEL=8, MIN_CELL=289, MIN_WHITE=1156, REPEATS=1, r_med=15.98


Post-processing TIFF:  20%|██        | 2/10 [01:45<06:57, 52.18s/frame]

...

[INFO] KERNEL=8, MIN_CELL=272, MIN_WHITE=1088, REPEATS=1, r_med=15.50


Post-processing TIFF:  90%|█████████ | 9/10 [08:16<00:55, 55.28s/frame]

[INFO] KERNEL=8, MIN_CELL=282, MIN_WHITE=1128, REPEATS=1, r_med=15.79


Post-processing TIFF: 100%|██████████| 10/10 [09:15<00:00, 55.51s/frame]



✅ Saved 0..3 TIFF:  /content/drive/MyDrive/Final_Project/output_predictions/post_processing_out/post_processed_0_3.tif
✅ Saved 0/85/170/255 TIFF: /content/drive/MyDrive/Final_Project/output_predictions/post_processing_out/post_processed_0_255.tif
✅ Saved NPZ (0..3 frames): /content/drive/MyDrive/Final_Project/output_predictions/post_processing_out/frames_0_3_list.npz
